In [ ]:
import cv2
import numpy as np 
import pandas as pd 

import keras
from keras import regularizers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Add
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.losses import categorical_crossentropy
from sklearn.metrics import accuracy_score
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [ ]:
def row2image(row):
    pixels, emotion = row['pixels'], emotion_map[row['emotion']]
    img = np.array(pixels.split(), dtype=np.uint8)  # Convert pixel values to numpy array
    img = img.reshape(48, 48)
    image = np.stack((img, img, img), axis=-1)  # Stack the grayscale image to form a 3-channel image
    return image, emotion


In [ ]:
#Axes to compare number of emotions per dataset
def setup_axe(axe,df,title):
    df['emotion'].value_counts(sort=False).plot(ax=axe, kind='bar', rot=0)
    axe.set_xticklabels(emotion_labels)
    axe.set_xlabel("Emotions")
    axe.set_ylabel("Number")
    axe.set_title(title)
    
    # set individual bar lables using above list
    for i in axe.patches:
        # get_x pulls left or right; get_height pushes up or down
        axe.text(i.get_x()-.05, i.get_height()+120, \
                str(round((i.get_height()), 2)), fontsize=14, color='dimgrey',
                    rotation=0)

In [ ]:
def CRNO(df, dataName):
    df['pixels'] = df['pixels'].apply(lambda pixel_sequence: [int(pixel) for pixel in pixel_sequence.split()])
    data_X = np.array(df['pixels'].tolist(), dtype='float32').reshape(-1,width, height,1)/255.0   
    data_Y = to_categorical(df['emotion'], num_classes)  
    print(dataName, "_X shape:", data_X.shape, dataName, "_Y shape:", data_Y.shape)
    #print(data_X) #[[[0.27450982]
    #print(data_Y) #[[1. 0. 0. ... 0. 0. 0.]
    return data_X, data_Y

In [ ]:
def plot_training(results):
    fig, axes = plt.subplots(1,2, figsize=(18, 6))
    # Plot training & validation accuracy values
    axes[0].plot(results.history['acc'])
    axes[0].plot(results.history['val_acc'])
    axes[0].set_title('Model accuracy')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    axes[1].plot(results.history['loss'])
    axes[1].plot(results.history['val_loss'])
    axes[1].set_title('Model loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
##Functions

#Plotting confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
        #print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize=(12,6))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

# early stopping
es = EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights=True)

In [ ]:
data = pd.read_csv('../Dataset/Dataset/CK+/ckextended.csv')
#check data shape
data.shape

In [ ]:
data.head(5)

In [ ]:
data.Usage.value_counts()

In [ ]:
data.emotion.value_counts()

In [ ]:
emotion_map = {0: 'Angry', 1: 'Digust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral', 7:'Contempt'}
emotion_counts = data['emotion'].value_counts(sort=False).reset_index()
emotion_counts.columns = ['emotion', 'number']
emotion_counts['emotion'] = emotion_counts['emotion'].map(emotion_map)
emotion_counts

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x=emotion_counts['emotion'], y=emotion_counts['number'])
plt.title('Class distribution')
plt.ylabel('Number', fontsize=12)
plt.show()

In [ ]:
distinct_values = data['emotion'].unique()
print(distinct_values)

In [ ]:
data['emotion'] = data['emotion'].rank(method='dense').astype(int) - 1

In [ ]:
plt.figure(0, figsize=(16,10))
for i in range(1,9):
    face = data[data['emotion'] == i-1].iloc[0]
    img = row2image(face)
    plt.subplot(2,4,i)
    plt.imshow(img[0])
    plt.title(img[1])

plt.show()  

In [ ]:
data = data[~data['emotion'].isin([1, 2, 5, 7])] 

In [ ]:
data.emotion.value_counts()

In [ ]:
emotion_map = {0: 'Angry', 3: 'Happy', 4: 'Sad', 6: 'Neutral'}

emotion_mapping = {0: 0, 3: 1, 4: 2, 6: 3}

data.loc[:, 'emotion'] = data['emotion'].map(emotion_mapping).astype(np.int32)

In [ ]:
data.emotion.value_counts()

In [ ]:
df_majority = data[data['emotion'] == 3]
df_minority_1 = data[data['emotion'] == 1]
df_minority_0 = data[data['emotion'] == 0]
df_minority_2 = data[data['emotion'] == 2]

df_majority_downsampled = df_majority.sample(n=max(len(df_minority_1), len(df_minority_0), len(df_minority_2)),
                                              random_state=42)

df_balanced = pd.concat([df_majority_downsampled, df_minority_1, df_minority_0, df_minority_2])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_balanced.emotion.value_counts()

In [ ]:
data_train = df_balanced[df_balanced['Usage']=='Training'].copy()
data_val = df_balanced[df_balanced['Usage'] == 'PublicTest'].copy()
data_test = df_balanced[df_balanced['Usage'] == 'PrivateTest'].copy()
print("train shape: {}, \nvalidation shape: {}, \ntest shape: {}".format(data_train.shape, data_val.shape, data_test.shape))

In [ ]:
data_train.emotion.value_counts()

In [ ]:
def row3image(row, emotion_map):
    pixels, emotion = row['pixels'], emotion_map[row['emotion']]
    img = np.array(pixels.split(), dtype=np.uint8)  # Convert pixel values to numpy array
    img = img.reshape(48, 48)
    image = np.stack((img, img, img), axis=-1)  # Stack the grayscale image to form a 3-channel image
    return image, emotion

In [ ]:
emotion_map = {0: 'Angry',  1: 'Happy', 2: 'Sad', 3: 'Neutral'}

plt.figure(0, figsize=(16,10))
for i in range(4):  # Adjust range to match emotion indices
    face = data_train[data_train['emotion'] == i].iloc[0]  # Use i directly without subtracting 1
    img = row3image(face, emotion_map)
    plt.subplot(2, 4, i+1)  # Increment i by 1 to start subplot index from 1
    plt.imshow(img[0])
    plt.title(img[1])

plt.show()

In [ ]:
emotion_labels = ['Angry', 'Happy', 'Sad', 'Neutral']

fig, axes = plt.subplots(1,3, figsize=(20,8), sharey=True)
setup_axe(axes[0],data_train,'train')
setup_axe(axes[1],data_val,'validation')
setup_axe(axes[2],data_test,'test')
plt.show()

In [ ]:
num_classes = 4
width, height = 48, 48
num_epochs = 100
batch_size = 32

train_X, train_Y = CRNO(data_train, "train") #training data
val_X, val_Y     = CRNO(data_val, "val") #validation data
test_X, test_Y   = CRNO(data_test, "test") #test data

In [ ]:
from tensorflow.keras.models import load_model

base_model = load_model('./Model/FER_v3.h5')

In [ ]:
base_model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
base_model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
                    metrics=['accuracy'])

In [ ]:
history = base_model.fit(data_generator.flow(train_X, train_Y, batch_size),
                            steps_per_epoch=len(train_X) / batch_size,
                            epochs=num_epochs,
                            verbose=2, 
                            callbacks = [es],
                            validation_data=(val_X, val_Y))

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
test_true = np.argmax(test_Y, axis=1)
test_pred = np.argmax(base_model.predict(test_X), axis=1)
vgg_16_score = accuracy_score(test_true, test_pred)
print("1.VGG-16 CNN Model Accuracy on test set: {:.4f}".format(accuracy_score(test_true, test_pred)))

In [ ]:
plot_confusion_matrix(test_true, test_pred, classes=emotion_labels, normalize=True, title='Normalized confusion matrix')
plt.show()

In [ ]:
base_model.save('../Recognition/Model/FER_v3_w_CK+.h5')

In [ ]:
def row2image(row):
    pixels, emotion = row['pixels'], emotion_map[row['emotion']]
    img = np.array(pixels.split(), dtype=np.uint8)  # Convert pixel values to numpy array
    img = img.reshape(48, 48)
    image = np.stack((img, img, img), axis=-1)  # Stack the grayscale image to form a 3-channel image
    return image, emotion


In [ ]:
def CRNO(df, dataName):
    df['pixels'] = df['pixels'].apply(lambda pixel_sequence: [int(pixel) for pixel in pixel_sequence.split()])
    data_X = np.array(df['pixels'].tolist(), dtype='float32').reshape(-1,width, height,1)/255.0   
    data_Y = to_categorical(df['emotion'], num_classes)  
    print(dataName, "_X shape:", data_X.shape, dataName, "_Y shape:", data_Y.shape)
    #print(data_X) #[[[0.27450982]
    #print(data_Y) #[[1. 0. 0. ... 0. 0. 0.]
    return data_X, data_Y

In [ ]:
##Functions

#Plotting confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    #else:
        #print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize=(12,6))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
data = pd.read_csv('../Dataset/Dataset/CK+/ckextended.csv')
#check data shape
data.shape

In [ ]:
data.Usage.value_counts()

In [ ]:
emotion_map = {0: 'Angry', 1: 'Digust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral', 7:'Contempt'}
emotion_counts = data['emotion'].value_counts(sort=False).reset_index()
emotion_counts.columns = ['emotion', 'number']
emotion_counts['emotion'] = emotion_counts['emotion'].map(emotion_map)
emotion_counts

In [ ]:
distinct_values = data['emotion'].unique()
print(distinct_values)

In [ ]:
plt.figure(0, figsize=(16,10))
for i in range(1,9):
    face = data[data['emotion'] == i-1].iloc[0]
    img = row2image(face)
    plt.subplot(2,4,i)
    plt.imshow(img[0])
    plt.title(img[1])

plt.show()  

In [ ]:
data.emotion.value_counts()

In [ ]:
data.emotion.value_counts()

In [ ]:
df_balanced.emotion.value_counts()

In [ ]:
data_train.emotion.value_counts()

In [26]:
data_train

,emotion,pixels,Usage
0,3,9 10 10 7 9 6 7 12 22 19 22 22 21 23 22 21 16 ...,Training
1,0,191 202 78 44 37 42 35 31 45 73 68 76 79 96 16...,Training
2,0,64 66 70 72 67 35 18 10 20 37 65 78 104 127 15...,Training
3,1,79 85 82 31 19 16 14 15 13 20 13 6 2 4 31 92 1...,Training
7,2,168 172 187 182 185 168 154 95 151 253 210 135...,Training
...,...,...,...
204,2,254 255 63 56 55 56 70 81 83 70 66 67 49 60 69...,Training
205,1,75 52 19 23 28 30 24 22 14 8 28 8 24 63 87 161...,Training
206,1,9 1 0 10 8 2 0 1 5 4 1 0 0 0 15 40 58 104 138 ...,Training
208,1,48 44 25 25 29 32 34 27 17 13 28 62 116 143 16...,Training


In [ ]:
emotion_map = {0: 'Angry',  1: 'Happy', 2: 'Sad', 3: 'Neutral'}

plt.figure(0, figsize=(16,10))
for i in range(4):  # Adjust range to match emotion indices
    face = data_train[data_train['emotion'] == i].iloc[0]  # Use i directly without subtracting 1
    img = row3image(face, emotion_map)
    plt.subplot(2, 4, i+1)  # Increment i by 1 to start subplot index from 1
    plt.imshow(img[0])
    plt.title(img[1])

plt.show()

In [ ]:
num_classes = 4
width, height = 48, 48
num_epochs = 100
batch_size = 32

train_X, train_Y = CRNO(data_train, "train") #training data
val_X, val_Y     = CRNO(data_val, "val") #validation data
test_X, test_Y   = CRNO(data_test, "test") #test data

In [ ]:
base_model.summary()

In [ ]:
base_model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
                    metrics=['accuracy'])

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plot_confusion_matrix(test_true, test_pred, classes=emotion_labels, normalize=True, title='Normalized confusion matrix')
plt.show()